In [7]:
import os
import sys
sys.path.insert(0, "/home/gstupp/projects/metaproteomics/")
#BASE = os.path.join(os.path.dirname(os.path.realpath(sys.argv[0])), "..")
BASE = "/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples"
DATA = os.path.join(BASE, "data")
OUT = os.path.join(BASE, "out")

from metaproteomics import utils
import pandas as pd
from itertools import chain
import re
import shelve
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from collections import Counter
from collections import defaultdict
from pymongo import MongoClient

from metaproteomics.analysis import build_loci ,functional_analysis
from metaproteomics.analysis.DBInfo import DBInfo

# ssh -fNL 27028:localhost:27018 gbwl
# ssh -fNL 27027:localhost:27017 gbwl
client = MongoClient(port=27028)
protDB = client['ProtDB_072114']['ProtDB_072114']
domainDB = client['DomainDB_072114']['DomainDB_072114']
hashDB = client['HashDB_072114']['HashDB_072114']
f = functional_analysis.Functionizer(protDB=protDB, hashDB=hashDB, domainDB=domainDB)

db_info = DBInfo("compil_mgm_localhost")
metadata = build_loci.read_metadata("metadata.csv")

In [28]:
pd.options.display.max_colwidth = 120

In [29]:
metadata.T

,sample_type,sample_name,path
H1,healthy,H1,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H1.txt
H2,healthy,H2,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H2.txt
H3,healthy,H3,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H3.txt
H4,healthy,H4,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H4.txt
H5,healthy,H5,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H5.txt
H6,healthy,H6,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H6.txt
H7,healthy,H7,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H7.txt
H8,healthy,H8,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/H8.txt
UC1,uc,UC1,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/UC1.txt
UC2,uc,UC2,/home/gstupp/projects/Wolan/triflic_BI/UnenrichedTriflic-BISamples/data/UC2.txt


In [9]:
#%% Parse samples
samples = shelve.open(os.path.join(OUT,"samples.shelve"))
for sample_name, sample_info in tqdm(list(metadata.iteritems())):
    sample = build_loci.Sample(sample_name, sample_info.path, db_info, sample_info)
    samples[sample.sample_name] = sample


  0%|          | 0/18 [00:00<?, ?it/s]
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 18/18 [05:08<00:00, 17.12s/it]


In [10]:
# Build protein clusters for each sample
protein_clusters = shelve.open(os.path.join(OUT,"protein_clusters.shelve"))
for name,sample in samples.items():
    protein_clusters[name] = sample.build_protein_clusters()

In [11]:
# generate proteins clusters across all samples
sample_pep_quant = {sample.sample_name:sample.pep_quant for sample in samples.values()}
grouped_loci = build_loci.group_across_samples(list(chain(*protein_clusters.values())), sample_pep_quant)
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci.pkl.gz"), force=True)

load obo file /home/gstupp/projects/metaproteomics/metaproteomics/goatools/go-basic.obo
48437 nodes imported


In [12]:
# write out matrix
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df.csv"))

In [13]:
df.shape

(18, 25394)

In [14]:
grouped_loci = utils.load(os.path.join(OUT,"grouped_loci.pkl.gz"))

In [33]:
# filter loci, min spectral count of 4 in at least two samples
min_quant = 4
min_samples = 8
grouped_loci = [locus for locus in grouped_loci if len([x for x in locus.quantification.values() if x>=min_quant]) >= min_samples]
len(grouped_loci)

5420

In [31]:
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt.pkl.gz"), force=True)

In [34]:
df = build_loci.to_df(grouped_loci, norm=False)
df.T.to_csv(os.path.join(OUT,"df_filt.csv"))
df.shape

(18, 5420)

In [35]:
df

,202956,209005,374028,407882,645785,678725,734086,757818,811098,900503,...,168146554,168149697,168181307,168187138,168199724,168211078,168226647,168235166,168237364,168249633
H1,13,25,29,27,9,0,10,14,39,15,...,5,5,4,5,4,13,13,0,9,8
H2,5,31,6,5,17,7,5,7,0,23,...,6,3,1,9,1,8,17,0,16,14
H3,10,27,19,7,10,10,3,4,0,12,...,5,2,2,7,7,5,7,4,9,8
H4,7,0,11,6,16,1,3,4,0,0,...,7,4,6,14,7,5,12,9,16,11
H5,4,8,0,0,25,10,0,0,8,0,...,5,5,7,8,13,4,7,7,15,9
H6,6,24,5,4,5,9,4,3,9,0,...,5,4,18,7,0,7,7,8,8,14
H7,8,10,0,0,30,10,0,5,7,13,...,13,6,6,10,10,9,34,9,18,11
H8,2,46,0,6,1,13,6,0,12,6,...,5,3,11,16,1,6,12,0,10,5
UC1,13,35,19,8,20,7,4,4,42,9,...,6,6,11,1,6,14,27,5,16,8
UC11,0,13,4,29,0,0,11,0,25,180,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# run functional annotation
for locus in tqdm(grouped_loci):
    locus.annotate()
utils.save(grouped_loci, os.path.join(OUT,"grouped_loci_filt_annot.pkl.gz"), force=True)

100%|██████████| 14618/14618 [1:25:29<00:00,  2.85it/s]


In [22]:
#%% mark as human/mouse and other
from metaproteomics.analysis import taxonomy
#t = taxonomy.Taxonomy(host="wl-cmadmin.scripps.edu")
t = taxonomy.Taxonomy(host="localhost", port=27027)
# human or mouse taxids and ancestors (up to the phylum chordata (7711)):
chordata = set(t.taxid_to_taxonomy(7711)['lineage'])
human = set(t.taxid_to_taxonomy(9606)['lineage']) - chordata
mouse = set(t.taxid_to_taxonomy(10090)['lineage']) - chordata
human_mouse = human | mouse
for locus in grouped_loci:
    locus.human_mouse = True if locus.lca in human_mouse else False

In [23]:
#%% Build a locus -> metadata table
for locus in grouped_loci:
    names = [x['d'] for x in locus.prot_info]
    gn=set(chain(*[re.findall("GN=([^ ]*)",name) for name in names]))
    locus.gn = ','.join(gn) if gn else ''
    locus.gn1 = list(gn)[0] if len(gn)==1 else ''
locus_df = pd.DataFrame({locus.cluster_id: {'name': locus.name, 'human_mouse': locus.human_mouse, 'lca':locus.lca, 'gn':locus.gn, 'gn1':locus.gn1} for locus in grouped_loci}).T
locus_df.to_csv(os.path.join(OUT,"locus_df.csv"))